In [2]:
from SantaFe import SantaFeAgent, SantaFeFood, SantaFeWorld
from SantaFe.Grammar import terminals
from grammaticalevolutiontools.evolution import cross_over, mutation

from tqdm import tqdm
import random
import heapq

%matplotlib tk

In [3]:
AGENT_CMAP = {SantaFeAgent: 'red'}
OBJ_CMAP = {SantaFeFood: 'lime'}

In [10]:
world = SantaFeWorld()
best_agent = None
best_score = -1

num_generations = 1
num_agents = 1000
num_to_keep = 500
num_to_regen = 200
num_ticks = 100

agents = [SantaFeAgent() for _ in range(num_agents)]
best_agents = []
best_agent = None

for gen in range(num_generations):
    print(f"GENERATION {gen + 1}: ")
    best_agents.clear()
    print('running agents...')
    for agent in tqdm(agents):
        world.load_new_agent(agent)
        world.tick(num_ticks)

        if len(best_agents) < num_to_keep:
            heapq.heappush(best_agents, (agent.score + 1, agent))
        else:
            heapq.heapreplace(best_agents, (agent.score + 1, agent))

        if not best_agent or agent.score > best_agent.score:
            best_agent = agent
            print(f'new best agent: {best_agent.score}')

    print('creating new generation...')
    new_agents = [SantaFeAgent() for _ in range(num_to_regen)]

    while len(new_agents) < num_agents:
        weights, agents = zip(*best_agents)
        agent1, agent2 = random.choices(agents, weights=weights, k=2)
        prog1, prog2 = cross_over.cross_over(agent1.program, agent2.program)
        
        if random.random() < 0.7:
            mutation.mutate_terminals(prog1, num_mutations=2, terminal_types=terminals)
        if random.random() < 0.7:
            mutation.mutate_terminals(prog2, num_mutations=2, terminal_types=terminals)

        new_agents.append(SantaFeAgent(prog1))
        new_agents.append(SantaFeAgent(prog2))

    agents = new_agents


print(best_agent.score)
print(best_agent.program)

GENERATION 1: 
running agents...


  0%|          | 5/1000 [00:00<00:21, 45.77it/s]

new best agent: 0
new best agent: 3.0


  2%|▏         | 20/1000 [00:00<00:24, 39.77it/s]

new best agent: 11.0


100%|██████████| 1000/1000 [00:32<00:00, 30.80it/s]


creating new generation...
11.0
if_food_ahead(Move, if_wall_ahead(Right, if_food_ahead(Move, if_wall_ahead(Left, Move, dist=1), dist=3), dist=2), dist=3)


In [12]:
world.load_new_agent(best_agent, recording_on=True)
world.tick(num_ticks)
anim = world.generate_animation(
    bg_color='black', 
    agent_colors=AGENT_CMAP, 
    obj_colors=OBJ_CMAP,
    arrow_color='blue'
    )

In [14]:
anim.play(pause=50)

In [13]:
best_agent

(SantaFeAgent, world=SantaFeWorld, program='if_food_ahead(Move, if_wa...', pos=(23, 1), dir=UP, rewards=11.0)